# Metaprogramming
* superficially, metaclasses are not difficult to understand
* details can get complicated, don't abuse them
* code is harder to read

# The __new__ method
* Static method not bound to the object

In [11]:
class Person:
    def __init__(self,name):
        self.name = name
    def __repr__(self):
        return f"Person(name='{self.name}')"
p = object.__new__(Person)
p.__init__('john')

print(p)

Person(name='john')


# override __new__

In [13]:
class Point:
    def __new__(cls,x,y):
        print('creating instance...',x,y)
        instance = super().__new__(cls)
        return instance
    def __init__(self,x,y):
        print('Init called...',x,y)
        self.x=x
        self.y=y
    
p = Point(1,2)

creating instance... 1 2
Init called... 1 2


In [16]:
class Person:
    def __new__(cls,name):
        print(f'Person: instantiating {cls.__name__}...')
        instance = object.__new__(cls)
        return instance
    
    def __init__(self,name):
        print('Person: initializing instance...')
        self.name = name

class Student(Person):
    def __new__(cls,name,major):
        print(f'Student: instantiating {cls.__name__}...')
        instance = super().__new__(cls,name)
        return instance
    def __init__(self,name,major):
        print('Student: initializing instance...')
        super().__init__(name)
        self.major = major

p = Person('john')
s = Student('kim', "Major")

Person: instantiating Person...
Person: initializing instance...
Student: instantiating Student...
Person: instantiating Student...
Student: initializing instance...
Person: initializing instance...


# How classes are created
* A class is an instance of type
* type is a callable and it is a class itself and inherits from object.
* type(class_name,class_bases,class_dict):
    * class_name: name of the class
    * class_bases: class to inherit from
    * class_dict: class dictionary

In [20]:
import math
class Circle(object):
    def __init__(self,x,y,r):
        self.x = x
        self.y = y
        self.r = r
    def area(self):
        return math.pi*self.r**2

print('Circle' in globals())
print(type(circle))

True
<class 'type'>


# emulate class creation

In [31]:
class_name = 'Circle'
class_body ="""     
def __init__(self,x,y,r):
    self.x = x
    self.y = y
    self.r = r
def area(self):
    return math.pi*self.r**2 
        """
class_bases = ()
class_dict = {}

exec(class_body,globals(),class_dict)
print(class_dict)

Circle = type(class_name,class_bases,class_dict)
print(type(Circle), '\n')
c = Circle(0,0,1)
print(c.__dict__)
print(c.area())

{'__init__': <function __init__ at 0x00000269C6210288>, 'area': <function area at 0x00000269C62104C8>}
<class 'type'> 

{'x': 0, 'y': 0, 'r': 1}
3.141592653589793


In [48]:
# inheriting from type
class CustomType(type):
    def __new__(cls,name,bases,class_dict):
        print('customized type creation!')
        class_dict['circ'] = lambda self: 2*math.pi * self.r
        cls_obj = super().__new__(cls,name,bases,class_dict)
        return cls_obj

class_body = """ 
def __init__(self,x,y,r):
    self.x = x
    self.y = y
    self.r = r
def area(self):
    return math.pi*self.r**2 
             """
class_dict = {}
exec(class_body,globals(),class_dict)
Circle = CustomType('Circle',(),class_dict)
print(type(Circle))
print(isinstance(Circle, type))
c = Circle(0,0,1)
print(c.area())
print(c.circ())
Circle.__dict__

customized type creation!
<class '__main__.CustomType'>
True
3.141592653589793
6.283185307179586


mappingproxy({'__init__': <function __main__.__init__(self, x, y, r)>,
              'area': <function __main__.area(self)>,
              'circ': <function __main__.CustomType.__new__.<locals>.<lambda>(self)>,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Circle' objects>,
              '__weakref__': <attribute '__weakref__' of 'Circle' objects>,
              '__doc__': None})

# Metaclasses
* by default python uses type as the Metaclasses
    * class Person(metaclass=type)
* but we can override this:
    * class Person(metaclass=MyType)

In [51]:
import math
class CustomType(type):
    def __new__(mcls,name,bases,class_dict):
        print(f'using custom metaclass: {mcls} to create class: {name}')
        cls_obj = super().__new__(mcls,name,bases,class_dict)
        cls_obj.circ = lambda self: 2*math.pi * self.r
        return cls_obj

class Circle(metaclass=CustomType):
    def __init__(self,x,y,r):
        self.x = x
        self.y = y
        self.r = r
    def area(self):
        return math.pi*self.r**2

c = Circle(0,0,1)
print(c.area())
print(c.circ())
Circle.__dict__

using custom metaclass: <class '__main__.CustomType'> to create class: Circle
3.141592653589793
6.283185307179586


mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Circle.__init__(self, x, y, r)>,
              'area': <function __main__.Circle.area(self)>,
              '__dict__': <attribute '__dict__' of 'Circle' objects>,
              '__weakref__': <attribute '__weakref__' of 'Circle' objects>,
              '__doc__': None,
              'circ': <function __main__.CustomType.__new__.<locals>.<lambda>(self)>})

# Class Decorators
* used for create, delete or modify class attributes
* modify methods


In [56]:
def savings(cls):
    cls.account_type = 'savings'
    return cls
def checking(cls):
    cls.account_type = 'checking'
    return cls

class Account:
    pass

@savings
class Bank1Savings(Account):
    pass
@savings
class Bank2Savings(Account):
    pass
@checking
class Bank1Checking(Account):
    pass
@checking
class Bank2Checking(Account):
    pass

print(Bank1Checking.__dict__)
print(Bank2Checking.__dict__)


{'__module__': '__main__', '__doc__': None, 'account_type': 'checking'}
{'__module__': '__main__', '__doc__': None, 'account_type': 'checking'}


In [58]:
# adding attribute to a class
def account_type(type_):
    def decorator(cls):
        cls.account_type = type_
        return cls
    return decorator

@account_type('Savings')
class Bank1Savings(Account):
    pass
@account_type('Checking')
class Bank1Checking(Account):
    pass

print(Bank1Checking.__dict__)
print(Bank2Checking.__dict__)

{'__module__': '__main__', '__doc__': None, 'account_type': 'Checking'}
{'__module__': '__main__', '__doc__': None, 'account_type': 'checking'}


In [61]:
# adding a method to a class
def hello(cls):
    cls.hello = lambda self: f'{self} says hello!'
    return cls
@hello
class Person:
    def __init__(self,name):
        self.name=name
    def __str__(self):
        return self.name
print(vars(Person), '\n')
p = Person('john')
p.hello()

{'__module__': '__main__', '__init__': <function Person.__init__ at 0x00000269C4F701F8>, '__str__': <function Person.__str__ at 0x00000269C4F70168>, '__dict__': <attribute '__dict__' of 'Person' objects>, '__weakref__': <attribute '__weakref__' of 'Person' objects>, '__doc__': None, 'hello': <function hello.<locals>.<lambda> at 0x00000269C4F70AF8>} 



'john says hello!'

In [2]:
# logger for debugging
from functools import wraps
def func_logger(fn):
    @wraps(fn)
    def inner(*args,**kwargs):
        result = fn(*args,**kwargs)
        print(f'Log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

@class_logger
class Person:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name}, and I am {self.age} years old'
        
p = Person('john', 29)
p.greet()

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet
Log: Person.__init__((<__main__.Person object at 0x000002770AF89B88>, 'john', 29), {}) = None
Log: Person.greet((<__main__.Person object at 0x000002770AF89B88>,), {}) = Hello, my name is john, and I am 29 years old


'Hello, my name is john, and I am 29 years old'

In [ ]:
# Decorator Classes
* opposite to Class Decorator
* here we want to use a class to decorate functions

In [12]:
from types import MethodType
class Logger:
    def __init__(self,fn):
        self.fn = fn
    def __call__(self,*args,**kwargs):
        print(f'Log: {self.fn.__name__} called.')
        return self.fn(*args,**kwargs)
    def __get__(self,instance,owner_class):
        print(f'__get__ called: self={self}, instance={instance}')
        if instance is None:
            print('\t returning self unbound...')
            return self
        else:
            print('\t returning self as method bound to instance...')
            return MethodType(self,instance)
class Person:
    def __init__(self,name):
        self.name = name
    @Logger
    def say_hello(self):
        return f'{self.name} says hello!'

@Logger
def say_hello():
    pass


say_hello()
p = Person('Alex')
print('\n')
p.say_hello()

Log: say_hello called.


__get__ called: self=<__main__.Logger object at 0x000002770D023148>, instance=<__main__.Person object at 0x000002770E47A688>
	 returning self as method bound to instance...
Log: say_hello called.


'Alex says hello!'

# Metaclass Parameters


In [16]:
class MyMetaclass(type):
    def __new__(mcls,name,bases,cls_dict, **extra_attrs):
        print('Creating class with some extra attributes:', extra_attrs)
        cls_dict.update(extra_attrs)
        new_cls = super().__new__(mcls,name,bases,cls_dict)
        return new_cls

class Account(metaclass=MyMetaclass, account_type = 'savings', apr=0.5):
    pass

print(vars(Account))

Creating class with some extra attributes: {'account_type': 'savings', 'apr': 0.5}
{'__module__': '__main__', 'account_type': 'savings', 'apr': 0.5, '__dict__': <attribute '__dict__' of 'Account' objects>, '__weakref__': <attribute '__weakref__' of 'Account' objects>, '__doc__': None}


# The __prepare__ Method

In [17]:
class MyMeta(type):
    def __new__(mcls,name,bases,cls_dict,**kwargs):
        cls_dict.update(kwargs)
        return super().__new__(mcls,name,bases,cls_dict)

class MyClass(metaclass=MyMeta,arg1=100,arg2=200):
    pass

vars(MyClass)

mappingproxy({'__module__': '__main__',
              'arg1': 100,
              'arg2': 200,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

In [18]:
# By using __prepare__ the cls_dict is created automatically.
class MyMeta(type):
    def __prepare__(name,bases,**kwargs):
        return kwargs
    def __new__(mcls,name,bases,cls_dict,**kwargs):
        return super().__new__(mcls,name,bases,cls_dict)

class MyClass(metaclass=MyMeta,arg1=100,arg2=200):
    pass

vars(MyClass)

mappingproxy({'arg1': 100,
              'arg2': 200,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

# The Attribute Read Accesor

In [19]:
class Person:
    def __getattr__(self,name):
        alt_name = '_' + name
        print(f'could not find {name}, trying {alt_name}...')

        try:
            return super().__getattr__(alt_name)
        except AttributeError:
            raise AttributeError(f'could not find {name} or {alt_name}')
p=Person()
try:
    p.age
except AttributeError as ex:
    print(type(ex).__name__, ex)

could not find age, trying _age...
AttributeError could not find age or _age


# The Attribute Write Accesor

In [23]:
class MyNonDataDesc:
    def __get__(self,instance,owner_class):
        print('__get__ called non data descriptor')

class MyDataDesc:
    def __set__(self,instance,value):
        print('__set__ called on data descriptor')
    def __get__(self,instance,owner_class):
        print('__get__ called data descriptor')

class MyClass:
    non_data_desc = MyNonDataDesc()
    data_desc = MyDataDesc()
    def __setattr__(self,name,value):
        print('__setattr__ called')
        super().__setattr__(name,value)

m = MyClass()
print(m.__dict__)
m.data_desc = 100
m.non_data_desc = 200
print(m.__dict__)


{}
__setattr__ called
__set__ called on data descriptor
__setattr__ called
{'non_data_desc': 200}
